## This notebook reads output.sub files from multiple SWAT directories and produces a summary table of their water and sediment budgets 

In [72]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import zipfile

### Paths

In [73]:
# Datapath of raw unzipped files
datapath = os.path.join("..", 'Raw_data', 'Unzipped')

results_path = os.path.join("..", 'Results')

### Produce lists of models and scenarios under each one

In [6]:
# generate sceanario paths
list_of_models = []
list_of_scenarios = []
list_of_sub = []
list_of_sed = []
pathlist = []

# path to scenarios is datapath, copy_mod, model_name, 'Scenarios'

for copy_mod in os.listdir(datapath):
    print('\n {}'.format(copy_mod))
    path_a = os.path.join(datapath, copy_mod)
    
    model_name = next(os.walk(path_a))[1]
    #print(model_name)
    
    path_b = os.path.join(datapath, copy_mod, model_name[0], 'Scenarios')
    scenario_list = next(os.walk(path_b))[1]
    print(scenario_list)
    
    for scenario in scenario_list:        
        # to write to dataframe later
        list_of_scenarios.append(scenario)
        list_of_models.append(model_name[0])
        
        path_c = os.path.join(datapath, copy_mod, model_name[0], 'Scenarios', scenario, 'TxtInOut')
        #print(path_c)
        
        list_of_sub.append(os.path.exists(os.path.join(path_c, 'output.sub')))
        list_of_sed.append(os.path.exists(os.path.join(path_c, 'output.sed')))
        pathlist.append(path_c)
        
        
summary_Frame = pd.DataFrame({"model":list_of_models, "scenario":list_of_scenarios, 
                              "Has_output_sub":list_of_sub  , "Has_output_sed":list_of_sed, 
                             "Full_rel_path":pathlist})

# summary_Frame.to_csv(os.path.join("..", "List_of_scenarios.csv"))

summary_Frame


 
 Copy of EastWestMaui_Green
['Default', 'EastWestMaui_rch_RCP45_Fog', 'EastWestMaui_rch_RCP45_nondiv_Fog', 'EastWestMaui_rch_RCP85_Fog', 'EastWestMaui_rch_RCP85_nondiv_Fog']

 
 Copy of EastWestMaui_Growth
['Default', 'EastWestMaui_rch_RCP45_Fog', 'EastWestMaui_rch_RCP45_nondiv_Fog', 'EastWestMaui_rch_RCP85_Fog', 'EastWestMaui_rch_RCP85_nondiv_Fog', 'Kahakuloa_rch_20yr_CcGrowth']

 
 Copy of EastWestMaui_MGrowth
['Default', 'EastWestMaui_rch_RCP45_Fog', 'EastWestMaui_rch_RCP45_nondiv_Fog', 'EastWestMaui_rch_RCP85_Fog', 'EastWestMaui_rch_RCP85_nondiv_Fog']

 
 Copy of Hanawi
[' Hana_rch_Sim2', 'Default', 'Hanawi_sub_Sim1', 'Hanawi_sub_Sim2', 'Hana_rch_Sim1', 'Hana_rch_Sim2', 'H_rch_1990to2009', 'H_rch_1995', 'H_rch_2004']

 
 Copy of Honokohau_V2
['Default', 'Honokohau_rch_yr_RainTPT']

 
 Copy of Honolua
['Default', 'Honolua_rch_2006to2009', 'Honolua_rch_2006to2009_div', 'Honolua_rch_2006to2014_BW', 'Honolua_rch_Sim1_2006to2009_Fog', 'Honolua_rch_Sim1_2006to2014_BW', 'Honolua_rch_Si

# Catergorize output files into two lists. 
Note that about 90 of them were set to write outputs on a yearly timestep, and 50 of them were set to output data on a daily timestep, I am not sure why. 

kind of long run time

In [9]:


# names Swat gives to output.sub
colnames_SUB = ['something', 'SUB',  'GIS',  'MON',  'AREAkm2',  'PRECIPmm', 'SNOMELTmm',    'PETmm',    'ETmm', 'SWmm', 'PERCmm',   'SURQmm',   
            'GW_Qmm',   'WYLDmm',   'SYLDt/ha', 'ORGNkg/ha',    'ORGPkg/ha',    'NSURQkg/ha',   'SOLPkg/ha',    'SEDPkg/ha',    
            'LATQ(mm)', 'LATNO3kg/ha',  'GWNO3kg/ha',   'CHOLAmic/L',   'CBODUmg/L',    'DOXQmg/L', 'TNO3kg/ha',    'QTILEmm',  'TVAPkg/ha']

yr_list  = []
dayly_list =  []
wtflist = []

for scen in pathlist:                            # for every scenario 
    OUTFILE = os.path.join(scen, "output.sub")

    # Open up the output.sub as a txt in memory
    listo = []
    with open(OUTFILE) as csv_file:                         
        csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
        line_count = 0
        for row in csv_reader:
            listo.append(row)
        data = listo[9:]        # cut off the headder line junk

    # Deal with SWAT's junky buggieness
    for i in data:
        splitit = i[3].split(".")               # this little blok is because SWAT messed up the Mon column and stuck it onto the area col, why?
        i[3] = splitit[0]
        i.insert(4, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...
  
    # Send the output file to a dataframe for pandas bliss
    df = pd.DataFrame(data, columns=colnames_SUB)
    
# Hla, why are some models output on a yearly resolution and why are some out on a daily resolution??? 

    # if its a ryearly resolution model... 
    if df['MON'].unique()[-1] == '20':      # The Mon 20 is the annual average of all 20 years, all models seem to be 1990-2010
        yr_list.append(scen)
        
    # otherwise its a daily output...
    elif df['MON'].unique()[-1] in  ['366', '365']:     # If the last entry in the unique values of the mon column are...
        dayly_list.append(scen)
        
    # and apparently there are a couple outliers??    
    else: 
        wtflist.append(scen) 

..\Raw_data\Unzipped\Copy of EastWestMaui_Green\EastWestMaui_Green\Scenarios\Default\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of EastWestMaui_Green\EastWestMaui_Green\Scenarios\EastWestMaui_rch_RCP45_Fog\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of EastWestMaui_Green\EastWestMaui_Green\Scenarios\EastWestMaui_rch_RCP45_nondiv_Fog\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of EastWestMaui_Green\EastWestMaui_Green\Scenarios\EastWestMaui_rch_RCP85_Fog\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001'

yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of Hanawi\Hanawi\Scenarios\Hanawi_sub_Sim1\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of Hanawi\Hanawi\Scenarios\Hanawi_sub_Sim2\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of Hanawi\Hanawi\Scenarios\Hana_rch_Sim1\TxtInOut
Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60

yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of Honokohau_V2\Honokohau_V2\Scenarios\Honokohau_rch_yr_RainTPT\TxtInOut
yearly
['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '20']
..\Raw_data\Unzipped\Copy of Honolua\Honolua\Scenarios\Default\TxtInOut
Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102'

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

...

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

Daily
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '158' '159' '160'
 '161' '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172'
 '173' '174' '175' '176' '177' '178' '179' '180' '181' '1

## Now actually extract data, 

the run time for the yearly models is reasonable 

In [14]:
# extract data from yearly sub files 

interestedparams_mm = ['PRECIPmm','ETmm', 'PERCmm', 'SURQmm', 'LATQ(mm)', 'GW_Qmm']        
interestedparams_ha = [ 'SYLDt/ha' ]

final_frame = pd.DataFrame()   # empty frame to contain results

for idx, scen in enumerate(yr_list):
    OUTFILE = os.path.join(scen, "output.sub")
    Scen_name = "{}--{}".format(scen.split("\\")[-4], scen.split("\\")[-2] )
    print(Scen_name)

    listo = []
    with open(OUTFILE) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
        line_count = 0
        for row in csv_reader:
            listo.append(row)
        data = listo[9:]        # cut off the headder line BS


    for i in data:
        splitit = i[3].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
        i[3] = splitit[0]
        i.insert(4, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

    df = pd.DataFrame(data, columns=colnames_SUB)
    Ave_frame = df[df['MON'] == "20"]
    Ave_frame = Ave_frame.reset_index(drop=True)    # make index start at 1
    cols = Ave_frame.columns
    Ave_frame[cols] = Ave_frame[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings 
    
    area_m2 = Ave_frame['AREAkm2']*1000000
    area_ha = Ave_frame['AREAkm2']*100
    
    ColList = ["Scenario"]; Val_Lst = [Scen_name]
    for i in interestedparams_mm: 
        p = sum(((Ave_frame[i]*0.001)*area_m2)/365)     # scale by area and convert the mm values to m3/day of water  and ADD up all the water in each subbasin
        colname = i[:-2]+"_m3pd"                        # rename the column to reflect new units
        ColList.append(colname)  ; Val_Lst.append(p)


    for j in interestedparams_ha:
        m = sum(((area_ha*Ave_frame[j])/365))                # scale by area and convert the mm values to m3/day of water  and ADD up all the water in each subbasin
        colname = j[:-3]+"_Tons_per_d"                       # rename the column to reflect new units
        ColList.append(colname) ; Val_Lst.append(m)
        
        
    Summary_frameSUB = pd.DataFrame([Val_Lst], columns=ColList )
    
    # also add a total area column for double checks    
    area_sum = sum(Ave_frame['AREAkm2'])
    Summary_frameSUB.insert(1, 'AREAkm2', area_sum)
    
    
    final_frame = final_frame.append(Summary_frameSUB)
    
final_frame.to_csv(os.path.join("..", "From_Yearly_WB_components.csv"))

final_frame

EastWestMaui_Green--Default
EastWestMaui_Green--EastWestMaui_rch_RCP45_Fog
EastWestMaui_Green--EastWestMaui_rch_RCP45_nondiv_Fog
EastWestMaui_Green--EastWestMaui_rch_RCP85_Fog
EastWestMaui_Green--EastWestMaui_rch_RCP85_nondiv_Fog
EastWestMaui_Growth--Default
EastWestMaui_Growth--EastWestMaui_rch_RCP45_Fog
EastWestMaui_Growth--EastWestMaui_rch_RCP45_nondiv_Fog
EastWestMaui_Growth--EastWestMaui_rch_RCP85_Fog
EastWestMaui_Growth--EastWestMaui_rch_RCP85_nondiv_Fog
EastWestMaui_MGrowth--Default
EastWestMaui_MGrowth--EastWestMaui_rch_RCP45_Fog
EastWestMaui_MGrowth--EastWestMaui_rch_RCP45_nondiv_Fog
EastWestMaui_MGrowth--EastWestMaui_rch_RCP85_Fog
EastWestMaui_MGrowth--EastWestMaui_rch_RCP85_nondiv_Fog
Hanawi--Default
Hanawi--Hanawi_sub_Sim1
Hanawi--Hanawi_sub_Sim2
Hanawi--H_rch_1990to2009
Honokohau_V2--Default
Honokohau_V2--Honokohau_rch_yr_RainTPT
Honolua--Honolua_rch_yr_Fog
Honolua--Honolua_sub_1990to2014_BW
Honolua--Honolua_sub_Sim1_SLRSPT&WnSp50pct
Honolua--Honolua_sub_Sim1_SLRSPT&WnSp50

,Scenario,AREAkm2,PRECIP_m3pd,ET_m3pd,PERC_m3pd,SURQ_m3pd,LATQ(m_m3pd,GW_Q_m3pd,SYLDt_Tons_per_d
0,EastWestMaui_Green--Default,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345
0,EastWestMaui_Green--EastWestMaui_rch_RCP45_Fog,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345
0,EastWestMaui_Green--EastWestMaui_rch_RCP45_non...,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345
0,EastWestMaui_Green--EastWestMaui_rch_RCP85_Fog,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738
0,EastWestMaui_Green--EastWestMaui_rch_RCP85_non...,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738
...,...,...,...,...,...,...,...,...,...
0,Wahikuli_MGrowth--Default,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773
0,Wahikuli_MGrowth--Wahikuli_rch_RCP45_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773
0,Wahikuli_MGrowth--Wahikuli_rch_RCP45_nondiv_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773
0,Wahikuli_MGrowth--Wahikuli_rch_RCP85_Fog,66.657996,259033.605542,116251.105141,10942.617634,33469.816619,99159.669630,9473.529221,5.534262


## Extract the Daily Files
Broken into 2 cells, first one has Very long run time

In [69]:
# extract data from Daily sub files into a dictionary for ease of time
data_dic = {}

for idx, scen in enumerate(dayly_list):
    OUTFILE = os.path.join(scen, "output.sub")
    Scen_name = "{}--{}".format(scen.split("\\")[-4], scen.split("\\")[-2] )
    print(Scen_name)
    
    listo = []
    with open(OUTFILE) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
        line_count = 0
        for row in csv_reader:
            listo.append(row)
        data = listo[9:]        # cut off the headder line BS
        
    for i in data:
        splitit = i[3].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
        i[3] = splitit[0]
        i.insert(4, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

    df = pd.DataFrame(data, columns=colnames_SUB)
    cols = df.columns
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not string
    
    data_dic[Scen_name] = df

EastWestMaui_Growth--Kahakuloa_rch_20yr_CcGrowth
Hanawi-- Hana_rch_Sim2
Hanawi--Hana_rch_Sim1
Hanawi--Hana_rch_Sim2
Hanawi--H_rch_1995
Hanawi--H_rch_2004
Honolua--Default
Honolua--Honolua_rch_2006to2009
Honolua--Honolua_rch_2006to2009_div
Honolua--Honolua_rch_2006to2014_BW
Honolua--Honolua_rch_Sim1_2006to2009_Fog
Honolua--Honolua_rch_Sim1_2006to2014_BW
Honolua--Honolua_rch_Sim2_1990to2014_BW
Honolua--Honolua_rch_Sim2_2006to2009_div
Honolua--Honolua_rch_Sim2_2006to2009_Fog
Honolua--Honolua_rch_Sim3_2006to2009_div
Honopou--Default
Honopou--H_rch_Sim1
Honopou--H_rch_Sim2
J2017_EastWestMaui--EastWestMaui_rch_Sim1_RainTPT
J2017_EastWestMaui--EastWestMaui_rch_Sim1_SLRSPT&WnSp50pct
J2017_EastWestMaui--EastWestMaui_rch_Sim2_RainTPT
J2017_EastWestMaui--EastWestMaui_rch_WQ1_SLR&WnSp50pct
J2017_EastWestMaui--EastWestMaui_rch_WQ2_SLR&WnSp50pct
Kalena--Default
Kalena--K_rch_Sim1
Kalena--K_rch_Sim2
Lahaina--Lahaina_rch_Sim1
North_CentralMaui--Default
North_CentralMaui--NCM_rch_Sim1
North_CentralMaui

In [74]:
# Consolidate each giant df into a yearly average 

interestedparams_mm = ['PRECIPmm','ETmm', 'PERCmm', 'SURQmm', 'LATQ(mm)', 'GW_Qmm'] 
interestedparams_ha = [ 'SYLDt/ha']
Final_cols = ['AREAkm2', 'PRECIP_m3pd', 'ET_m3pd', 'PERC_m3pd', 'SURQ_m3pd', 'LATQ(m_m3pd', 'GW_Q_m3pd', 'SYLDt_Tons_per_d']

#Create empty container
final_frame_D = pd.DataFrame()

for seeneo in data_dic.keys():

    # Aggregate all rows by subbasin 
    group_by_SUB_frame = data_dic[seeneo].groupby('SUB').mean()

    for param in interestedparams_mm:
        new_name = param[:-2]+"_m3pd"   #make new column headding 
        group_by_SUB_frame[new_name]= (group_by_SUB_frame[param]*0.001)*(group_by_SUB_frame['AREAkm2']*1000000)

    for param in interestedparams_ha:
        new_name = param[:-3]+"_Tons_per_d"   #make new column headding 
        group_by_SUB_frame[new_name]= (group_by_SUB_frame[param])*(group_by_SUB_frame['AREAkm2']*100)


    tempframe = pd.DataFrame(group_by_SUB_frame[Final_cols].sum(axis=0)).T       # Sum all subbasins into a whole model value and keep it in dataframe format (col wize necessitates the T. 
    
    tempframe.insert(0, 'Scenario', seeneo)

    final_frame_D = final_frame_D.append(tempframe)
    
final_frame_D.to_csv(os.path.join(results_path, "From_Daily_WB_components.csv"))   

### Merge the yearly and the daily models into one frame for summarization

In [76]:
final_frame['run_rez'] = "yearly"
final_frame_D['run_rez'] = "Daily"

Conbined_final_frame = final_frame.append(final_frame_D)

Conbined_final_frame.sort_values("Scenario", inplace=True)

# Commented out because will save a version with % diff attached later. 
# Conbined_final_frame.to_csv(os.path.join("..", "All_scenarios.csv"))

Conbined_final_frame

,Scenario,AREAkm2,PRECIP_m3pd,ET_m3pd,PERC_m3pd,SURQ_m3pd,LATQ(m_m3pd,GW_Q_m3pd,SYLDt_Tons_per_d,run_rez
0,EastWestMaui_Green--Default,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly
0,EastWestMaui_Green--EastWestMaui_rch_RCP45_Fog,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly
0,EastWestMaui_Green--EastWestMaui_rch_RCP45_non...,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly
0,EastWestMaui_Green--EastWestMaui_rch_RCP85_Fog,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738,yearly
0,EastWestMaui_Green--EastWestMaui_rch_RCP85_non...,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738,yearly
...,...,...,...,...,...,...,...,...,...,...
0,Wahikuli_MGrowth--Default,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly
0,Wahikuli_MGrowth--Wahikuli_rch_RCP45_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly
0,Wahikuli_MGrowth--Wahikuli_rch_RCP45_nondiv_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly
0,Wahikuli_MGrowth--Wahikuli_rch_RCP85_Fog,66.657996,259033.605542,116251.105141,10942.617634,33469.816619,99159.669630,9473.529221,5.534262,yearly


# Assess the % difference between different scenarios for each of the watersheds

### i don't actually know which scenarios are supposed to be the base case, here I assume its the "Default" ones 

Another question is are there multiple defaults for the same watershed, i.e. a daily default, and a yearly default? 

In [78]:
# List of the individual models
model_list = Conbined_final_frame['Scenario'].apply(lambda x: x.split('--')[0]).unique()

# create a dataframe of only the default models 
bases_frame = pd.DataFrame()
for mod in model_list:
    def_one = Conbined_final_frame[Conbined_final_frame['Scenario'] == f"{mod}--Default"]
    bases_frame = bases_frame.append(def_one)
    
bases_frame = bases_frame.reset_index(drop=True)
#del bases_frame['run_rez']
base_model_list = bases_frame['Scenario'].apply(lambda x: x.split('--')[0]).unique()

# Also check if any models are missing the default scenario 
Models_missing_Default_scenario = list(set(model_list) - set(base_model_list))
print(f"These models are missing a Defualt Scenario {Models_missing_Default_scenario}")

# get ready to break each model area (watershed) into a frame of individual scenarios and calculate the % difference from each one
pct_diff_cols =  ['Scenario', 'AREA_pctdif', 'PRECIP_pctdif', 'ET_pctdif', 'PERC_pctdif', 'SURQ_pctdif', 'LATQ_pctdif', 'GW_Q_pctdif', 'SYLDt_pctdif'] 
pct_diff_frame = pd.DataFrame()

for i in base_model_list:
    base_row = bases_frame[bases_frame['Scenario'].apply(lambda x: x.split('--')[0]) == i]             # Pull out the row that matches the model we are working on in this iteration
    base_row.set_index('Scenario', inplace=True)
    del base_row['run_rez']
    base_row = base_row.apply(pd.to_numeric, errors='coerce', axis=1)                                  # make the columns numeric not strings 
    
    scenarios_frame_tmp = Conbined_final_frame[Conbined_final_frame['Scenario'].apply(lambda x: x.split('--')[0]) == i]   # Pull out the scenarios in the model we are working on in this iteration
    scenarios_frame_tmp.set_index('Scenario', inplace=True)
    del scenarios_frame_tmp['run_rez']
    scenarios_frame_tmp = scenarios_frame_tmp.apply(pd.to_numeric, errors='coerce', axis=1)                               # make the columns numeric not strings 
    
    basevals = base_row.values                  # array of the base case values
    senvals  = scenarios_frame_tmp.values       # array of the scenario case values
    tmp_idx = scenarios_frame_tmp.index         # Pull out the index to save for later  

    bit_frame = pd.DataFrame((senvals - basevals)/basevals, columns=base_row.columns)     # perform the subtraction as an array, then send back to dataframe format
    bit_frame.set_index(tmp_idx, inplace = True)                                              # re-label each scenario 
    bit_frame.reset_index(drop=False, inplace=True)                                           # Set label as a colum
    bit_frame.columns = pct_diff_cols                                                         # rename columns
    
    pct_diff_frame = pct_diff_frame.append(bit_frame)     # Consolidate the % diff frames for each model into one big dataframe
    
# Consolidate and save results    
Result_Frame = Conbined_final_frame.merge(pct_diff_frame, on="Scenario", how='outer')   # stick the % diff numbers onto the absolute magnitude numbers
Result_Frame.to_csv(os.path.join(results_path, "Final_Result_frame.csv"))

Result_Frame

These models are missing a Defualt Scenario ['J2017_EastWestMaui']


C:\Users\cshuler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\cshuler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


,Scenario,AREAkm2,PRECIP_m3pd,ET_m3pd,PERC_m3pd,SURQ_m3pd,LATQ(m_m3pd,GW_Q_m3pd,SYLDt_Tons_per_d,run_rez,AREA_pctdif,PRECIP_pctdif,ET_pctdif,PERC_pctdif,SURQ_pctdif,LATQ_pctdif,GW_Q_pctdif,SYLDt_pctdif
0,EastWestMaui_Green--Default,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,EastWestMaui_Green--EastWestMaui_rch_RCP45_Fog,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,EastWestMaui_Green--EastWestMaui_rch_RCP45_non...,144.489277,833044.766848,244112.967767,63216.814769,141360.140562,346684.852082,52624.001137,2.485345,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,EastWestMaui_Green--EastWestMaui_rch_RCP85_Fog,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738,yearly,0.0,-0.025883,-0.033627,-0.034718,-0.020769,-0.024332,-0.032276,0.005389
4,EastWestMaui_Green--EastWestMaui_rch_RCP85_non...,144.489277,811483.276159,235904.118134,61022.080628,138424.290391,338249.195149,50925.501028,2.498738,yearly,0.0,-0.025883,-0.033627,-0.034718,-0.020769,-0.024332,-0.032276,0.005389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Wahikuli_MGrowth--Default,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
136,Wahikuli_MGrowth--Wahikuli_rch_RCP45_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
137,Wahikuli_MGrowth--Wahikuli_rch_RCP45_nondiv_Fog,66.657996,266377.947261,119227.720049,11686.074726,32604.941931,103625.257731,10145.368213,7.740773,yearly,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
138,Wahikuli_MGrowth--Wahikuli_rch_RCP85_Fog,66.657996,259033.605542,116251.105141,10942.617634,33469.816619,99159.669630,9473.529221,5.534262,yearly,0.0,-0.027571,-0.024966,-0.063619,0.026526,-0.043094,-0.066221,-0.285051
